In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.transforms import Normalize, ToTensor, Compose, Resize
from torchvision import datasets
from torch.utils.data import DataLoader, Subset

In [4]:
data_transforms = {
    'train': Compose([
        Resize((28, 28)),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': Compose([
        Resize((28, 28)),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dir = 'archive/train'
test_dir = 'archive/test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train']) #100 000 pictures
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms['test'])

#chooses a random subset of 1000 pictures so that training is much faster
np.random.seed(42)
subset_indices = np.random.choice(len(train_dataset), 1000, replace=False)
train_dataset_subset = Subset(train_dataset, subset_indices)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#train_loader = DataLoader(train_dataset_subset, batch_size=32, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [5]:
class ImageDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            #I include 3 because each pixel is RGB
            nn.Linear(28*28*3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = ImageDiscriminator()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(params=model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

In [7]:
num_epochs = 3 #number of times it will update the NN
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad() #reset all weights
        outputs = model(images) 
        loss = criterion(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [1/3], Loss: 0.4366
Epoch [2/3], Loss: 0.3878
Epoch [3/3], Loss: 0.3843


In [8]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        predicted = torch.round(torch.sigmoid(outputs.squeeze()))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total} %')

Accuracy of the model on the test images: 81.935 %
